In [3]:
import os
from django.conf import settings

In [4]:
BASE_DIR = os.path.join('.')

In [5]:
settings.configure(
    BASE_DIR=BASE_DIR,
    INSTALLED_APPS=[
        'django.contrib.admin',
        'django.contrib.auth',
        'django.contrib.contenttypes',
        'django.contrib.sessions',
        'django.contrib.messages',
        'django.contrib.staticfiles',

        # 'rest_framework',
        # 'rest_framework.authtoken',
        # 'corsheaders',
        # 'django_extensions',

        'scores'
    ],
    DATABASES={
        'default': {
            'ENGINE': 'django.db.backends.sqlite3',
            'NAME': os.path.join(BASE_DIR, 'my_faces.sqlite3')
        }
    },
    MEDIA_URL='/media/',
    MEDIA_ROOT=os.path.join(BASE_DIR, 'media'),
    CACHES={
        'default': {
            'BACKEND': 'django.core.cache.backends.filebased.FileBasedCache',
            'LOCATION': os.path.join(BASE_DIR, 'cache'),
        }
    }

)

In [6]:
import django

In [7]:
os.environ.setdefault('DJANGO_ALLOW_ASYNC_UNSAFE', 'True')

'True'

In [8]:
django.setup()

In [9]:
from scores.models import Score

In [10]:
scores = Score.objects.all()

In [52]:
from django.db.models.aggregates import Avg, Sum, Count
from django.db.models import When, Case, Q
from scores.choices import POSITIVE_EMOTIONS, NEGATIVE_EMOTIONS
from scores.choices import SkinColor

In [53]:
logic1 = When(Q(emotion__in=POSITIVE_EMOTIONS), 1)
logic2 = When(Q(emotion__in=NEGATIVE_EMOTIONS), 0)
cases = Case(logic1, logic2)

In [56]:
# Glabal statistics
statistics = scores.annotate(average=Avg('numeric'), emotion_numeric=cases)

In [59]:
# By skin color
white_skintones = scores.filter(image__skin_color=SkinColor.WHITE)
black_skintones = scores.filter(image__skin_color=SkinColor.BLACK)


In [61]:
stats_white_skintones = white_skintones.annotate(average=Avg('numeric'), emotion_numeric=cases)
stats_black_skintones = white_skintones.annotate(average=Avg('numeric'), emotion_numeric=cases)

In [68]:
fields = ['id', 'image__image', 'average', 'emotion_numeric']
stat1 = statistics.values(*fields)
stat2 = stats_white_skintones.values(*fields)
stat3 = stats_black_skintones.values(*fields)

In [71]:
# Average score for all the website
global_average_score = scores.aggregate(Avg('numeric'))


In [81]:
# How emotions distribute by count globally
negative_emotions = scores.annotate(emotion_numeric=cases).filter(Q(emotion_numeric=0))
positive_emotions = scores.annotate(emotion_numeric=cases).filter(Q(emotion_numeric=1))

count_negative_emotions = negative_emotions.aggregate(Count('emotion_numeric'))
count_positive_emotions = positive_emotions.aggregate(Count('emotion_numeric'))

In [82]:
# How emotions distribute by skin tones
# negative_emotions1 = scores.filter(image__skin_tone=SkinColor.WHITE).annotate(emotion_numeric=cases).filter(Q(emotion_numeric=0))
# positive_emotions1 = scores.filter(image__skin_color=SkinColor.WHITE).annotate(emotion_numeric=cases).filter(Q(emotion_numeric=1))

# positive_emotions = scores.annotate(emotion_numeric=cases).filter(Q(emotion_numeric=1))

# count_negative_emotions = negative_emotions.aggregate(Count('emotion_numeric'))
# count_positive_emotions = positive_emotions.aggregate(Count('emotion_numeric'))

In [107]:
import pandas

In [126]:
df = pandas.DataFrame(data=scores.values('id', 'image__image', 'image__skin_color', 'numeric', 'emotion'))

In [127]:
df2 = df.rename(columns={'image__image': 'image', 'image__skin_color': 'skin_color'})

In [128]:
def transform_emotions(emotion):
    if emotion in POSITIVE_EMOTIONS:
        return 1
    else:
        return 0

df2['emotion_numeric'] = df2['emotion'].apply(transform_emotions)

In [129]:
df_scores = df2[['image', 'skin_color', 'numeric', 'emotion_numeric']]

In [130]:
df_scores

,image,skin_color,numeric,emotion_numeric
0,white\fw1.jpg,White,1,1
1,black\fw158.jpg,Black,3,0


In [149]:
df_scores.describe()

,numeric,emotion_numeric
count,2.000000,2.000000
mean,2.000000,0.500000
std,1.414214,0.707107
min,1.000000,0.000000
25%,1.500000,0.250000
50%,2.000000,0.500000
75%,2.500000,0.750000
max,3.000000,1.000000


In [186]:
scores_by_skin_tones = dict(df_scores.groupby('skin_color')['numeric'].mean())

stats = {
    'count': df_scores['numeric'].count(),
    'xxxx': None,
    'emotions': {
        'positive': df_scores[df_scores['emotion_numeric'] == 1]['numeric'].count(),
        'negative': df_scores[df_scores['emotion_numeric'] == 0]['numeric'].count(),
        'by_skin_tones': {
            'black': {
                'positive': None,
                'negative': None
            },
            'white': {
                'positive': None,
                'negative': None
            }
        }
    },
    'scores': {
        'average': df_scores['numeric'].mean(),
        'by_skin_tones': {
            'labels': list(scores_by_skin_tones.keys()),
            'datasets': [
                {
                    'label': 'Score by skin color',
                    'color': None,
                    'data': list(scores_by_skin_tones.values())
                }
            ]
        }
    }
}
stats


{'count': 2,
 'xxxx': None,
 'emotions': {'positive': 1,
  'negative': 1,
  'by_skin_tones': {'black': {'positive': None, 'negative': None},
   'white': {'positive': None, 'negative': None}}},
 'scores': {'average': 2.0,
  'by_skin_tones': {'labels': ['Black', 'White'],
   'datasets': [{'label': 'Score by skin color',
     'color': None,
     'data': [3.0, 1.0]}]}}}

In [199]:
df_scores['skin_color'] == 'Black'
df_scores['numeric'] == 1


0     True
1    False
Name: numeric, dtype: bool